In [2]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Load model and processor
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to("cuda")


/home/vasu/miniconda3/envs/rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8e07f63b20>>
Traceback (most recent call last):
  File "/home/vasu/miniconda3/envs/rl/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Load the CSV into a DataFrame
df = pd.read_csv("/content/drive/MyDrive/dataset/qna.csv")

# Display the first few rows
print(df.head())


In [ ]:
import os

# Step 2: Fix image paths

def fix_path(rel_path):
    rel_subpath = rel_path.replace("../dataset/", "")
    return os.path.join("/content/drive/MyDrive/dataset", rel_subpath)

df["fixed_path"] = df["image_path"].apply(fix_path)


In [ ]:
df

In [ ]:
# Track results
correct = 0
results = []

# Iterate through each row
for i, row in df.iterrows():
    try:
        image = Image.open(row["fixed_path"]).convert("RGB")
        question = row["question"]
        gt_answer = row["answer"].strip().lower()

        # VQA inference
        inputs = processor(images=image, text=question, return_tensors="pt").to("cuda", torch.float16)
        out = model.generate(**inputs)
        pred = processor.decode(out[0], skip_special_tokens=True).strip().lower()

        is_correct = pred == gt_answer

        results.append({
            "image_path": row["image_path"],
            "question": question,
            "ground_truth": gt_answer,
            "predicted": pred,
            "correct": is_correct
        })

        if is_correct:
            correct += 1

        print(f"Q: {question}\nGT: {gt_answer}\nPredicted: {pred}\n")

    except Exception as e:
        print(f"Error at row {i}: {e}")

# Final accuracy
accuracy = correct / len(df)
print(f"\nOverall Accuracy: {accuracy:.2%}")

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("/content/drive/MyDrive/dataset/vqa_results.csv", index=False)